In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Load data
data_folder = 'email_csv2'
all_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
df_list = []
for file in all_files:
    file_path = os.path.join(data_folder, file)
    df_temp = pd.read_csv(file_path)
    df_temp['source_file'] = file
    df_list.append(df_temp)

# Combine all DataFrames into one
df_all = pd.concat(df_list, ignore_index=True)
print("Total rows across all CSVs:", df_all.shape[0])
print("Total files read:", len(all_files))

Total rows across all CSVs: 161014
Total files read: 160


In [2]:
# Extract domain from emails
def extract_domain_from_email(email: str) -> str:
    if pd.isnull(email) or '@' not in str(email):
        return None
    email = str(email).replace('<', '').replace('>', '').strip()
    domain = email.split('@')[-1].split(':')[0].lower()
    return domain

df_all['domain'] = df_all['from'].apply(extract_domain_from_email)
df_domains = pd.DataFrame(sorted(df_all['domain'].dropna().unique()), columns=['domain'])
df_domains.to_csv('unique_domains.csv', index=False)
print("Saved to csv.")

Saved to csv.


In [3]:
import pandas as pd
import re
import requests
import time
import whois

df = pd.read_csv('domain_country.csv', encoding='ISO-8859-1')
df['domain'] = df['domain'].astype(str).str.strip().str.lower()
domains = df['domain'].dropna().unique()

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0'})

manual_map = {
    '3gpp.org': 'Unknown', 'chinamobile.com': 'China', 'bt.com': 'United Kingdom',
    'neclab.eu': 'Germany', 'email.mot.com': 'United States', 'nortelnetworks.com': 'Canada',
    'ac.upc.edu': 'Spain', 'sharplabs.com': 'Japan', 'hutchison3g.com': 'Hong Kong',
    'allot.com': 'Israel', 'ztetx.com': 'China', 'renesasmobile.com': 'Japan',
    'aol.com': 'United States', 'testingtech.com': 'Germany', 'imraff.com': 'India',
    'ec.europa.eu': 'Belgium'
}

tld_map = {
    '.cn': 'China', '.com.cn': 'China', '.net.cn': 'China', '.gov.cn': 'China',
    '.hk': 'Hong Kong', '.mo': 'Macau', '.tw': 'Taiwan', '.jp': 'Japan', '.co.jp': 'Japan',
    '.kr': 'South Korea', '.co.kr': 'South Korea', '.in': 'India', '.co.in': 'India',
    '.sg': 'Singapore', '.my': 'Malaysia', '.th': 'Thailand', '.vn': 'Vietnam',
    '.ph': 'Philippines', '.id': 'Indonesia', '.us': 'United States', '.uk': 'United Kingdom',
    '.co.uk': 'United Kingdom', '.gov.uk': 'United Kingdom', '.ie': 'Ireland', '.de': 'Germany',
    '.fr': 'France', '.it': 'Italy', '.es': 'Spain', '.pt': 'Portugal', '.nl': 'Netherlands',
    '.be': 'Belgium', '.se': 'Sweden', '.no': 'Norway', '.fi': 'Finland', '.dk': 'Denmark',
    '.ch': 'Switzerland', '.at': 'Austria', '.pl': 'Poland', '.cz': 'Czech Republic',
    '.sk': 'Slovakia', '.hu': 'Hungary', '.ro': 'Romania', '.bg': 'Bulgaria',
    '.si': 'Slovenia', '.hr': 'Croatia', '.lt': 'Lithuania', '.lv': 'Latvia',
    '.ee': 'Estonia', '.ru': 'Russia', '.by': 'Belarus', '.ua': 'Ukraine',
    '.kz': 'Kazakhstan', '.il': 'Israel', '.ir': 'Iran', '.tr': 'Turkey',
    '.sa': 'Saudi Arabia', '.ae': 'United Arab Emirates', '.qa': 'Qatar',
    '.jo': 'Jordan', '.lb': 'Lebanon', '.au': 'Australia', '.nz': 'New Zealand',
    '.ca': 'Canada', '.mx': 'Mexico', '.br': 'Brazil', '.ar': 'Argentina',
    '.cl': 'Chile', '.co': 'Colombia', '.pe': 'Peru', '.za': 'South Africa',
    '.ng': 'Nigeria', '.ke': 'Kenya', '.eg': 'Egypt', '.ma': 'Morocco', '.dz': 'Algeria',
    '.lu': 'Luxembourg', '.li': 'Liechtenstein', '.is': 'Iceland', '.gl': 'Greenland',
    '.su': 'Russia'
}

country_code_map = {
    'US': 'United States', 'CN': 'China', 'JP': 'Japan', 'KR': 'South Korea',
    'IN': 'India', 'GB': 'United Kingdom', 'UK': 'United Kingdom', 'DE': 'Germany',
    'FR': 'France', 'IT': 'Italy', 'ES': 'Spain', 'NL': 'Netherlands', 'SE': 'Sweden',
    'FI': 'Finland', 'CA': 'Canada', 'AU': 'Australia', 'AT': 'Austria',
    'CH': 'Switzerland', 'TW': 'Taiwan', 'RU': 'Russia', 'BR': 'Brazil', 'MX': 'Mexico',
    'AR': 'Argentina', 'IL': 'Israel', 'SG': 'Singapore', 'HK': 'Hong Kong',
    'MY': 'Malaysia', 'TH': 'Thailand', 'VN': 'Vietnam', 'PH': 'Philippines',
    'ID': 'Indonesia'
}

def guess_by_tld(domain):
    for tld, country in tld_map.items():
        if domain.endswith(tld):
            return country
    return None

def extract_company_name(domain):
    domain = re.sub(r'^(www\.|mail\.|email\.|smtp\.)', '', domain)
    parts = domain.split('.')
    if len(parts) >= 3 and parts[-2] in ['com', 'co', 'org', 'net', 'gov', 'edu']:
        return parts[-3].replace('-', ' ')
    elif len(parts) >= 2:
        return parts[-2].replace('-', ' ')
    return domain.replace('-', ' ')

def search_wikidata(company_name):
    try:
        params = {
            'action': 'wbsearchentities',
            'search': company_name,
            'language': 'en',
            'format': 'json',
            'type': 'item',
            'limit': 3
        }
        r = session.get("https://www.wikidata.org/w/api.php", params=params, timeout=10)
        data = r.json()
        for entity in data.get("search", []):
            if any(x in entity.get("description", "").lower() for x in ["company", "corporation", "business", "enterprise", "organization"]):
                return get_country_from_wikidata(entity['id'])
    except:
        return None
    return None

def get_country_from_wikidata(entity_id):
    try:
        url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbgetentities',
            'ids': entity_id,
            'format': 'json',
            'props': 'claims'
        }
        r = session.get(url, params=params, timeout=10)
        claims = r.json()['entities'][entity_id].get('claims', {})
        for pid in ['P17', 'P159', 'P495']:
            if pid in claims:
                country_id = claims[pid][0]['mainsnak']['datavalue']['value']['id']
                return get_country_label(country_id)
    except:
        return None
    return None

def get_country_label(country_id):
    try:
        url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbgetentities',
            'ids': country_id,
            'format': 'json',
            'props': 'labels'
        }
        r = session.get(url, params=params, timeout=10)
        return r.json()['entities'][country_id]['labels']['en']['value']
    except:
        return None

def whois_lookup(domain):
    try:
        w = whois.whois(domain)
        for field in ['country', 'registrant_country', 'admin_country', 'tech_country']:
            if hasattr(w, field):
                c = getattr(w, field)
                if isinstance(c, list):
                    c = c[0]
                if c:
                    c = c.strip().upper()
                    return country_code_map.get(c, c.title())
        for field in ['address', 'registrant_address', 'admin_address']:
            if hasattr(w, field):
                addr = getattr(w, field)
                if isinstance(addr, str):
                    for name in country_code_map.values():
                        if name.lower() in addr.lower():
                            return name
    except Exception as e:
        print(f"WHOIS failed for {domain}: {e}")
    return None

results = {}
for i, domain in enumerate(domains, 1):
    print(f"[{i}/{len(domains)}] {domain}")
    if domain in manual_map:
        results[domain] = manual_map[domain]
        continue
    country = guess_by_tld(domain)
    if not country:
        cname = extract_company_name(domain)
        country = search_wikidata(cname)
        time.sleep(1)
    if not country:
        country = whois_lookup(domain)
    results[domain] = country if country else 'Unknown'

df['country_filled'] = df['domain'].map(results)
df.to_csv("domain_country_with_whois.csv", index=False)
print("Saved to domain_country_with_whois.csv")


[1/1248] 10099.com.cn
[2/1248] 126.com
[3/1248] 163.com
[4/1248] 163.net
[5/1248] 189.cn
[6/1248] 263.net
[7/1248] 3genesis.com
[8/1248] 3gpp.org
[9/1248] 3in.org
[10/1248] 5gtbiitm.in
[11/1248] 5nord.de
[12/1248] 7layers.de
[13/1248] aal.siemens.dk
[14/1248] aalto.fi
[15/1248] aalyria.com
[16/1248] ac.upc.edu
[17/1248] accenture.com
[18/1248] acm.org
[19/1248] actcom.co.il
[20/1248] adare.de
[21/1248] admin.iscas.ac.cn
[22/1248] adobe.com
[23/1248] aegean.gr


2025-06-17 15:44:08,605 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


[24/1248] aenor.es
[25/1248] aeroflex.com
[26/1248] afnor.org
[27/1248] agd.gov.au
[28/1248] agentschaptelecom.nl
[29/1248] agentura-cas.cz
[30/1248] agilent.com
[31/1248] aioti.eu
[32/1248] airbiquity.com
[33/1248] airbus.com
[34/1248] aircom.co.za
[35/1248] airtel.com
[36/1248] airtel.es
[37/1248] ait.ac.at
[38/1248] ait.gr


2025-06-17 15:44:25,733 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


[39/1248] aj.jp.nec.com
[40/1248] alcatel-lucent.com
[41/1248] alcatel-lucent.fr
[42/1248] alcatel-sbell.com.cn
[43/1248] alcatel.com
[44/1248] alcatel.de
[45/1248] alcatel.fr
[46/1248] aldiscon.ie
[47/1248] alibaba-inc.com
[48/1248] alice.it
[49/1248] allgon.se
[50/1248] allianz.com
[51/1248] allot.com
[52/1248] alticelabs.com
[53/1248] am.exch.agilent.com
[54/1248] am1.ericsson.se
[55/1248] amazon.com
[56/1248] amdocs.com
[57/1248] anakyn.be
[58/1248] andrew.com
[59/1248] anemonetechnology.com
[60/1248] anet.com
[61/1248] anfr.fr
[62/1248] anite.com
[63/1248] anitetelecoms.com
[64/1248] anritsu.com
[65/1248] anterix.com
[66/1248] aol.com
[67/1248] ap.jp.nec.com
[68/1248] appcomsci.com
[69/1248] apple.com
[70/1248] applus.com
[71/1248] aqsacom.com
[72/1248] arib.or.jp
[73/1248] aricent.com
[74/1248] arm.com
[75/1248] arris.com
[76/1248] art.alcatel.fr
[77/1248] artemis-ia.eu
[78/1248] arthurslegal.com
[79/1248] ascertia.com
[80/1248] ascom.ch
[81/1248] asiainfo.com
[82/1248] asro.ro
[

...

[995/1248] spark.co.nz
[996/1248] spirent.com
[997/1248] spirentcom.com
[998/1248] sporton-lab.com
[999/1248] sporton.com.tw
[1000/1248] spreadtrum.com
[1001/1248] sprint.com
[1002/1248] sptvjsat.com
[1003/1248] srtc.org.cn
[1004/1248] ss.ts.fujitsu.co.jp
[1005/1248] ss8.com
[1006/1248] st.com
[1007/1248] standard.md
[1008/1248] starentnetworks.com
[1009/1248] starpointcomm.com
[1010/1248] statesoft.org
[1011/1248] stericsson.com
[1012/1248] stest.ch
[1013/1248] stewe.org
[1014/1248] strategictest.co.uk
[1015/1248] strict.nl
[1016/1248] student.uu.se
[1017/1248] students.tafe.tas.edu.au
[1018/1248] sunrisetelecom.com
[1019/1248] sutn.gov.sk
[1020/1248] swapcom.fr
[1021/1248] swisscom.com
[1022/1248] symantec.com
[1023/1248] symbian.com
[1024/1248] sympatico.ca
[1025/1248] synctechno.com
[1026/1248] synelixis.com
[1027/1248] systemdesign.it
[1028/1248] systemsoft.no
[1029/1248] sz.huawei.com.cn
[1030/1248] t-email.de
[1031/1248] t-mobil.de
[1032/1248] t-mobile.at
[1033/1248] t-mobile.co

2025-06-17 16:05:42,382 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


[1049/1248] team.telstra.com
[1050/1248] techmahindra.com
[1051/1248] technicolor.com
[1052/1248] tecnalia.com
[1053/1248] tecono.fi
[1054/1248] tejasnetworks.com
[1055/1248] tejet.cn
[1056/1248] tekcomms.com
[1057/1248] telcordia.com
[1058/1248] telcoware.com
[1059/1248] tele.pw.edu.pl
[1060/1248] teleca.com
[1061/1248] teleca.es
[1062/1248] telecanetworks.se
[1063/1248] telecom-sudparis.eu
[1064/1248] telecom.samsung.co.kr
[1065/1248] telecom.tno.nl
[1066/1248] telecomitalia.it
[1067/1248] telecomsys.com
[1068/1248] telefonica.com
[1069/1248] telefonica.es
[1070/1248] telekom.de
[1071/1248] telelogic.com
[1072/1248] telelogic.de
[1073/1248] telenor.com
[1074/1248] telenor.dk
[1075/1248] telia.se
[1076/1248] teliacompany.com
[1077/1248] teliasonera.com
[1078/1248] telit.com
[1079/1248] telital.com
[1080/1248] telitas.no
[1081/1248] telus.com
[1082/1248] temc2.com
WHOIS failed for temc2.com: No match for "TEMC2.COM".
>>> Last update of whois database: 2025-06-17T14:06:03Z <<<

NOTICE: 

2025-06-17 16:08:15,860 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed


[1173/1248] utimaco.com
[1174/1248] utstar.com
[1175/1248] utulsa.edu
[1176/1248] uwcc.org
[1177/1248] vads.vodafone.co.uk
[1178/1248] valyd.com
[1179/1248] vamansolutions.com
[1180/1248] vbtelecom.fr
[1181/1248] vde.com
[1182/1248] vdivde-it.de
[1183/1248] venturus.org.br
[1184/1248] verfassungsschutz.de
[1185/1248] verisign.com
[1186/1248] verizon.com
[1187/1248] verizon.net
[1188/1248] verizonwireless.com
[1189/1248] vf.vodafone.co.uk
[1190/1248] viasat.com
[1191/1248] viavisolutions.com
[1192/1248] videosystems.it
[1193/1248] vimatix.com
[1194/1248] virtualpalmtop.net
WHOIS failed for virtualpalmtop.net: No match for "VIRTUALPALMTOP.NET".
>>> Last update of whois database: 2025-06-17T14:08:35Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsorin